## Kristen Brandt 171482

https://discourse.igniterealtime.org/t/fail-to-start-openfire-4-3-0-on-macos-mojave-version-10-14-2/83863/5

In [24]:
import logging

from slixmpp import ClientXMPP
from slixmpp.exceptions import IqError, IqTimeout

#mensaje individual
<message to="user1@example.com" from="user2@example.com"> <body>Hi!</body>
</message>


#mensaje grupal
<message type="groupchat" to="room@conference.example.com"> <body>Hi everyone!</body>
</message>

#estado
<presence />
<presence from="user@example.com"> <show>away</show> <status>Getting lunch.</status> <priority>5</priority>
</presence>
<presence type="unavailable" />

In [25]:
from slixmpp.plugins.base import BasePlugin
from slixmpp.xmlstream.handler.callback import Callback
from slixmpp.xmlstream.matcher.xpath import MatchXPath
from slixmpp.xmlstream import ElementBase, ET, JID, register_stanza_plugin 
from slixmpp import Iq
import copy

In [26]:
class UserStore(object):
    def __init__(self): 
        
        self.users = {}
    def __getitem__(self, jid):
        return self.users.get(jid, None)
    
    def register(self, jid, registration):
        username = registration['username']
        
    def filter_usernames(user):
        return user != jid and self.users[user]['username'] == username
        conflicts = filter(filter_usernames, self.users.keys())
        if conflicts:
            return False 
        self.users[jid] = registration
        return True

    def unregister(self, jid):
        del self.users[jid]

In [27]:
class xep_0077(BasePlugin): #"""XEP-0077 In-Band Registration """
    
    def plugin_init(self):
        self.description = "In-Band Registration" 
        self.xep = "0077"
        self.form_fields = ('username', 'password') 
        self.form_instructions = ""
        self.backend = UserStore()
        self.xmpp.register_handler( 
            Callback('In-Band Registration',
                     MatchXPath('{%s}iq/{jabber:iq:register}query' % self.xmpp.default_ns),
                                self.__handleRegistration))
        register_stanza_plugin(Iq, Registration)
        
        
    def post_init(self):
        BasePlugin.post_init(self) 
        self.xmpp['xep_0030'].add_feature("jabber:iq:register")
        
        
    def __handleRegistration(self, iq): 
        if iq['type'] == 'get':
            # Registration form requested
            userData = self.backend[iq['from'].bare]
            self.sendRegistrationForm(iq, userData)
        elif iq['type'] == 'set':
                if iq['register']['remove']:
                    # Remove an account 
                    self.backend.unregister(iq['from'].bare)
                    self.xmpp.event('unregistered_user', iq) 
                    iq.reply().send()
                    return
                for field in self.form_fields:
                    if not iq['register'][field]: 
                        # Incomplete Registration
                        self._sendError(iq, '406', 'modify', 'not-acceptable', "Please fill in all fields.")
                        return
                if self.backend.register(iq['from'].bare, iq['register']): 
                    # Successful registration
                    self.xmpp.event('registered_user', iq)
                    reply = iq.reply()
                    reply.set_payload(iq['register'].xml)
                    reply.send() 
                else:
                # Conflicting registration     
                    self._sendError(iq, '409', 'cancel', 'conflict', "That username is already taken.")


    def setForm(self, *fields):
        self.form_fields = fields
        
        
        
    def setInstructions(self, instructions):
        self.form_instructions = instructions
        
        
    def sendRegistrationForm(self, iq, userData=None): 
        reg = iq['register']
        if userData is None:
            userData = {} 
        else:
            reg['registered'] = True 
        if self.form_instructions:
            reg['instructions'] = self.form_instructions
        
        for field in self.form_fields: 
            data = userData.get(field, '') 
            if data:
            # Add field with existing data
                reg[field] = data
            else:
                # Add a blank field
                reg.addField(field)
        reply = iq.reply()
        reply.set_payload(reg.xml) 
        reply.send()
        
        
    def _sendError(self, iq, code, error_type, name, text=''):
        reply = iq.reply() 
        reply.set_payload(iq['register'].xml)
        reply.error()
        reply['error']['code'] = code
        reply['error']['type'] = error_type 
        reply['error']['condition'] = name 
        reply['error']['text'] = text 
        reply.send()

In [28]:
## main menu
def main_menu():
    print("--------------------\nBienvenido a el chat de Kristen\n\n\tYa tiene un usuario? (1)\n\tDesea crear un usario nuevo? (2)\n\tSalir (3)")

In [ ]:

import ssl
import nest_asyncio


dele = True
opcion1 = 0
port = 5222


while dele:
    try:
        #displays main menu and ask user what option they want
        main_menu()
        opcion_general = int(input())
        if opcion_general == 1:
            #existing username
            opcion1 = 1
            dele = False
        elif opcion_general == 2:
            #new username
            opcion1 = 2
            dele = False
        elif opcion_general == 3:
            #get out out the chat
            opcion1 = 3
            dele = False
        else:
            #defensive programing
            print("Porfavor ingrese una opcion valida (1,2,3)\n")
    except:
        # defensive programing
        print("Porfavor ingrese una opcion valida (1,2,3)\n")
        

        #
# existing user
if opcion1 == 1:
    username = input("Porfavor ingrese su usuario: ")
    password = input("Porfavor ingrese su contraseña: ")
    
    xmpp = ClientXMPP(username, password)
    # ... Register plugins and event handlers ...
    xmpp.connect()
    xmpp.process()
    #verification for existing username
    
    
#new user    
elif opcion1 == 2:
    username_new = input("Porfavor ingrese el usuario que desea: ")
    password_new = input("Porfavor ingrese la contraseña que desea: ")
    
    #verification for existing username
    Registration.getRegistered()
    #if the username doesn't exist creates it
    #if the username exists error message
    
#get out of chat
else:
    print("\nGracias por utilizar el chat de Kristen\nSaliendo ...\n--------------------")
    
    

--------------------
Bienvenido a el chat de Kristen

	Ya tiene un usuario? (1)
	Desea crear un usario nuevo? (2)
	Salir (3)
1
Porfavor ingrese su usuario: kris@localhost
Porfavor ingrese su contraseña: kris


DEBUG    Loaded Plugin: RFC 6120: Stream Feature: STARTTLS
DEBUG    Loaded Plugin: RFC 6120: Stream Feature: Resource Binding
DEBUG    Loaded Plugin: RFC 3920: Stream Feature: Start Session
DEBUG    Loaded Plugin: RFC 6121: Stream Feature: Roster Versioning
DEBUG    Loaded Plugin: RFC 6121: Stream Feature: Subscription Pre-Approval
DEBUG    Loaded Plugin: RFC 6120: Stream Feature: SASL
DEBUG    Event triggered: connecting
DEBUG    DNS: Querying SRV records for localhost
DEBUG    DNS: Exception while querying for localhost SRV records: (12, 'Timeout while contacting DNS servers')
DEBUG    DNS: Querying localhost for AAAA records.
DEBUG    DNS: Exception while querying for localhost AAAA records: (12, 'Timeout while contacting DNS servers')
DEBUG    DNS: Querying localhost for A records.
DEBUG    DNS: Exception while querying for localhost A records: (12, 'Timeout while contacting DNS servers')
DEBUG    Event triggered: connected
DEBUG    SEND: <stream:stream to='localhost' xmlns:stream=

DEBUG    SEND:  
DEBUG    RECV: <iq type="get" id="13-164" from="localhost" to="admin@localhost/b65fjedo9"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="13-164" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    RECV: <iq type="get" id="813-165" from="localhost" to="kris@localhost/58en0kyp5p"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="813-165" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    RECV: <iq type="get" id="727-166" from="localhost" to="admin@localhost/8pml8uk759"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="727-166" to="localhost"><error type="canc

DEBUG    SEND: <iq type="error" id="798-187" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    RECV: <iq type="get" id="325-188" from="localhost" to="kris@localhost/58en0kyp5p"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="325-188" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    RECV: <iq type="get" id="213-189" from="localhost" to="admin@localhost/8pml8uk759"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="213-189" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers regis

DEBUG    RECV: <iq type="get" id="793-206" from="localhost" to="kris@localhost/58en0kyp5p"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="793-206" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    RECV: <iq type="get" id="502-207" from="localhost" to="admin@localhost/8pml8uk759"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="502-207" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    Scheduled event: Whitespace Keepalive
DEBUG    SEND:  
DEBUG    Scheduled event: Whitespace Keepalive
DEBUG    SEND:  
DEBUG    Scheduled event: Whitespace Keepalive
DEBUG    SEND:  
DEBUG    RECV: <iq type="ge

DEBUG    SEND: <iq type="error" id="596-237" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    RECV: <iq type="get" id="253-238" from="localhost" to="admin@localhost/8pml8uk759"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="253-238" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    Scheduled event: Whitespace Keepalive
DEBUG    SEND:  
DEBUG    Scheduled event: Whitespace Keepalive
DEBUG    SEND:  
DEBUG    Scheduled event: Whitespace Keepalive
DEBUG    SEND:  
DEBUG    RECV: <iq type="get" id="591-240" from="localhost" to="admin@localhost/b65fjedo9"><ping xmlns="urn:xmpp:ping" /></iq>
ERROR    Task was destroyed

DEBUG    SEND: <iq type="error" id="301-258" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    RECV: <iq type="get" id="706-259" from="localhost" to="kris@localhost/58en0kyp5p"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="706-259" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    RECV: <iq type="get" id="825-260" from="localhost" to="admin@localhost/8pml8uk759"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="825-260" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers regis

In [21]:
xmpp.abort()

DEBUG    Connection failed: [Errno 60] Connect call failed ('93.184.216.34', 5222)
DEBUG    Event triggered: connection_failed
DEBUG    Event triggered: reconnect_delay
DEBUG    Scheduled event: Whitespace Keepalive
DEBUG    SEND:  
DEBUG    Scheduled event: Whitespace Keepalive
DEBUG    SEND:  
DEBUG    RECV: <iq type="get" id="68-49" from="localhost" to="admin@localhost/6qpy5qv1mu"><ping xmlns="urn:xmpp:ping" /></iq>
DEBUG    SEND: <iq type="error" id="68-49" to="localhost"><error type="cancel"><feature-not-implemented xmlns="urn:ietf:params:xml:ns:xmpp-stanzas" /><text xmlns="urn:ietf:params:xml:ns:xmpp-stanzas">No handlers registered for this request.</text></error></iq>
DEBUG    End of stream received
DEBUG    Event triggered: killed
INFO     connection_lost: (None,)
DEBUG    Cancelling 0 slow send tasks
DEBUG    Event triggered: session_end
DEBUG    Event triggered: disconnected
DEBUG    End of stream received
DEBUG    Event triggered: killed
INFO     connection_lost: (None,)
DEB